In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

/home/codespace/miniforge3/envs/dsc80/lib/python3.12/site-packages/otter/export/__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# Lab 3 – Merging and Pivoting

## DSC 80, Fall 2025

### Due Date: Monday, October 20th at 11:59PM

## Instructions

Welcome to the third DSC 80 lab this quarter!

Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook, and **you will only submit that `lab.py` file**, not this notebook!

Some additional guidelines:
- **Unlike in DSC 10, labs will have both public tests and hidden tests.** The bulk of your grade will come from your scores on hidden tests, which you will only see on Gradescope after the assignment deadline.
- **Do not change the function names in the `lab.py` file!** The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name. If you changed something you weren't supposed to, you can find the original code in the [course GitHub repository](https://github.com/dsc-courses/dsc80-2025-wi).
- Notebooks are nice for testing and experimenting with different implementations before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file, since that's all you're submitting.
- You are encouraged to write your own additional helper functions to solve the lab, as long as they also end up in `lab.py`.

**To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.** More details on its usage are given at the bottom of this notebook.

**Importing code from `lab.py`**:

* Below, we import the `.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from lab import *

In [4]:
import os
import io
from pathlib import Path
import pandas as pd
import numpy as np

<div class="alert alert-block alert-danger" markdown="1">

**There are only two functions in this lab in which you may use a `for`-loop:**
- **`read_linkedin_survey` in Question 1.**
- **`read_student_surveys` in Question 2.**
    
**You may lose points if you use a `for`-loop or `while`-loop in any other question!**

</div>

## Part 1: Combining Data

### Question 1 – Making Connections 🤝

A group of students decided to send out a survey to their connections on LinkedIn. Each student asks 1000 of their connections for their first and last name, the company they currently work at, their job title, their email, and the university they attended.

**Your job is to combine all the data contained in the files `survey*.csv` (stored within the `data/responses` folder) into a single DataFrame. The number of files and the number of rows in each file may vary, so don't hardcode your answers!** To do so, implement the following two functions.

#### `read_linkedin_survey`

Complete the implementation of the function `read_linkedin_survey`, which takes in a Path for the directory where `survey*.csv` files are located and outputs a single DataFrame with six columns titled `'first name'`, `'last name'`, `'current company'`, `'job title'`, `'email'`, and `'university'` (in that order) containing the all of survey information from all the `.csv` files. Make sure to reset the index of the combined DataFrame before returning it so that the index is unique. When the function takes in an invalide directory it should raise a `FileNotFoundError`.

***Hints***:

- Take a look at a few of the files in the `responses` folder. You may have to do some data cleaning to combine the DataFrames!

- You can list the files in a directory using `.iterdir()`.

***Note***: Remember to use `Path()` to build file paths. Calling `.iterdir()` on a `Path()` object will automatically throw a `FileNotFoundError`.

<br>

#### `com_stats`

Complete the implementation of the function `com_stats`, which takes in a DataFrame returned by `read_linkedin_survey` and returns a list containing, in the following order: 
- The proportion of people who went to a university with the string `'Ohio'` in its name that have the string `'Programmer'` somewhere in their job title.
- The number of job titles that **end** with the exact string `'Engineer'`. Note that we're asking for the number of job titles, **not** the number of people!
- The job title that has the longest name (there are no ties).
- The number of people who have the word `'manager'` in their job title, uppercase or lowercase (`'Manager'`, `'manager'`, and `'mANAgeR'` should all count).

In [5]:
def read_linkedin_survey(dirname):
    """
    Liest alle Dateien 'survey*.csv' im Verzeichnis `dirname`, vereinheitlicht die Spalten
    und kombiniert sie in einen DataFrame mit Spalten:
    ['first name', 'last name', 'current company', 'job title', 'email', 'university'].
    Reset des Index vor Rückgabe. Ungültiger Pfad -> FileNotFoundError.
    """
    target_cols = ["first name", "last name", "current company", "job title", "email", "university"]

    def _clean_cols(df):
        # Spaltennamen normalisieren und auf Zielnamen mappen
        df = df.copy()

        def _norm(c):
            c = str(c).strip().lower()
            c = c.replace("-", " ").replace("_", " ")
            return " ".join(c.split())

        df.columns = [_norm(c) for c in df.columns]

        rename_map = {
            # first name
            "first name": "first name", "firstname": "first name", "first": "first name",
            # last name
            "last name": "last name", "lastname": "last name", "surname": "last name", "last": "last name",
            # current company
            "current company": "current company", "company": "current company",
            "company current": "current company", "current employer": "current company", "employer": "current company",
            # job title
            "job title": "job title", "title": "job title", "job": "job title", "position": "job title",
            # email
            "email": "email", "e mail": "email", "mail": "email",
            # university
            "university": "university", "school": "university", "alma mater": "university",
            "university attended": "university",
        }
        df = df.rename(columns={c: rename_map[c] for c in df.columns if c in rename_map})

        for col in target_cols:
            if col not in df.columns:
                df[col] = pd.NA

        return df[target_cols]

    # Path.iterdir() wirft FileNotFoundError bei ungültigem Pfad (gewünscht)
    dirpath = Path(dirname)
    files = [p for p in dirpath.iterdir() if p.is_file() and p.suffix.lower() == ".csv" and p.name.startswith("survey")]

    frames = []
    for p in files:
        df = pd.read_csv(p)
        frames.append(_clean_cols(df))

    if not frames:
        return pd.DataFrame(columns=target_cols).reset_index(drop=True)

    out = pd.concat(frames, ignore_index=True)
    return out.reset_index(drop=True)


def com_stats(df):
    """
    Erwartet den DataFrame aus read_linkedin_survey und gibt eine Liste zurück mit:
    [ proportion_ohio_programmer, n_unique_titles_ending_engineer, longest_title, n_people_manager ]

    1) Anteil der Personen, deren 'university' den String 'Ohio' (case-insensitive) enthält
       UND deren 'job title' die Zeichenkette 'Programmer' (case-sensitive) enthält.
       Wenn es keine 'Ohio'-Uni gibt -> 0.0.
    2) Anzahl *einzigartiger* Jobtitel, die exakt mit 'Engineer' enden (case-sensitive).
    3) Der Jobtitel mit der längsten Bezeichnung (keine Ties laut Aufgabe).
    4) Anzahl der Personen, deren Jobtitel das Wort 'manager' enthält (case-insensitive, als Wort).
    """
    uni = df["university"].astype("string")
    title = df["job title"].astype("string")

    # (1) Anteil Ohio & 'Programmer' (case-sensitive)
    ohio_mask = uni.str.contains("Ohio", case=False, na=False)
    programmer_mask = title.str.contains("Programmer", case=True, na=False)
    denom = int(ohio_mask.sum())
    num = int((ohio_mask & programmer_mask).sum())
    proportion = (num / denom) if denom > 0 else 0.0

    # (2) Anzahl einzigartiger Jobtitel, die mit 'Engineer' enden
    unique_titles = pd.Series(title.dropna().str.strip().unique(), dtype="string")
    n_end_engineer = int(unique_titles.apply(lambda t: str(t).endswith("Engineer")).sum())

    # (3) Längster Jobtitel
    cleaned = [str(t).strip() for t in title.dropna()]
    longest = max(cleaned, key=len) if cleaned else ""

    # (4) Personen mit Wort 'manager' (case-insensitive, Wortgrenzen)
    n_manager = int(title.str.contains(r"\bmanager\b", case=False, na=False, regex=True).sum())

    return [float(proportion), n_end_engineer, longest, n_manager]


In [6]:
# do not edit this cell -- it is needed for the tests
dirname = Path('data') / 'responses'
q1_out = read_linkedin_survey(dirname)
stats_out = com_stats(q1_out)

In [7]:
grader.check("q1")

q1 results: All test cases passed!

### Question 2 – Survey Says... 👨‍👩‍👧‍👦

Professor Aritra often sends out extra credit surveys asking students for their favorite animals, movies, and other favorite things. These surveys are stored in the `data/extra-credit-surveys` folder. Each file in that folder corresponds to a different survey question (except for `favorite1.csv`, which contains students' names and IDs).

Here's how extra credit works:
- Each student who has completed at least 50% of the survey questions receives 5 points of extra credit.
- If there is at least one survey question that at least 90% of the class answered (e.g. favorite animal), **everyone** in the class receives 1 point of extra credit. This overall class extra credit only applies twice, so if for example 95% of students answer the favorite color survey question and 91% answer the favorite animal survey question, and and 97% answer the favorite movie question, the entire class still receives 2 extra point as a class, not 3.
- Note that this means that the most extra credit any student can earn is 7 points.

#### `read_student_surveys`

Complete the implementation of the function `read_student_surveys` which takes in a Path describing the directory to a folder containing `favorite*.csv` files and outputs a DataFrame containing all of the survey data combined, indexed by student ID (a value 1-1000).

***Note***: Remember to use `Path()` to build file paths.


<br>

#### `check_credit`

Complete the implementation of the function `check_credit` which takes in a DataFrame returned by `read_student_surveys` and outputs a DataFrame indexed by student ID (a value 1-1000) with two columns:
- `'name'`, containing the name of each student, and
- `'ec'`, containing the number of extra credit points each student earned.

***Note***: For the genres survey, `'(no genres listed)'` does not count as a valid response for receiving extra credit. 

In [ ]:
def read_student_surveys(dirname):
    """
    Liest alle 'favorite*.csv' Dateien aus `dirname` und kombiniert sie zu einem
    DataFrame, der nach Studenten-ID (1..1000) indexiert ist.
    - 'favorite1.csv' enthält Name & ID -> Spalte 'name'
    - Alle anderen Dateien enthalten je eine Frage -> je eine Spalte (benannt nach dem Dateinamen-Stem)
    """
    dirpath = Path(dirname)  # wirft FileNotFoundError bei ungültigem Pfad bei iterdir()
    files = sorted([p for p in dirpath.iterdir()
                    if p.is_file() and p.suffix.lower() == ".csv" and p.name.startswith("favorite")],
                   key=lambda p: p.name.lower())

    # Hilfsfunktionen
    def _norm_cols(df):
        df = df.copy()
        def _n(c):
            c = str(c).strip().lower()
            c = c.replace("-", " ").replace("_", " ")
            return " ".join(c.split())
        df.columns = [_n(c) for c in df.columns]
        return df

    def _find_id_col(cols):
        for c in cols:
            if "id" == c or c.endswith(" id") or c.startswith("id"):
                return c
        # Fallback: erste Spalte namensähnlich zu 'id'
        for c in cols:
            if "id" in c:
                return c
        return None

    def _col_name_from_file(p):
        # Benenne Spalte nach Dateistamm, z.B. 'favorite_genres' -> 'favorite_genres'
        # und stelle sicher, dass 'genres' im Namen bleibt (wichtig für EC-Regel).
        stem = p.stem  # z.B. 'favorite_genres' oder 'favorite2'
        return stem.lower()

    # Beginne mit vollständiger ID-Referenz 1..1000
    out = pd.DataFrame(index=pd.Index(range(1, 1001), name="id"))

    for p in files:
        df = pd.read_csv(p)
        df = _norm_cols(df)
        id_col = _find_id_col(df.columns)
        if id_col is None:
            # Wenn keine ID-Spalte gefunden -> Überspringen (oder Fehler werfen)
            # Wir entscheiden uns für defensives Überspringen.
            continue

        if p.name.lower() == "favorite1.csv":
            # Suche nach 'name'
            name_col = None
            for c in df.columns:
                if "name" == c or c.endswith(" name") or c.startswith("name"):
                    name_col = c
                    break
            if name_col is None:
                # Fallback: wenn kein 'name' existiert, erstelle Dummy-Name
                tmp = df[[id_col]].copy()
                tmp["name"] = pd.NA
            else:
                tmp = df[[id_col, name_col]].rename(columns={id_col: "id", name_col: "name"})
            tmp["id"] = pd.to_numeric(tmp["id"], errors="coerce").astype("Int64")
            tmp = tmp.dropna(subset=["id"]).astype({"id": "int"})
            tmp = tmp.set_index("id")
            out = out.join(tmp, how="left")
        else:
            # Eine einzelne Frage/Antwort-Spalte extrahieren (alles außer ID/Name)
            candidates = [c for c in df.columns if c != id_col and c != "name"]
            if len(candidates) == 0:
                continue
            # Wenn mehrere Spalten, nimm die erste "nicht-leere" bevorzugt
            resp_col = candidates[0]
            col_name = _col_name_from_file(p)
            tmp = df[[id_col, resp_col]].rename(columns={id_col: "id", resp_col: col_name})
            tmp["id"] = pd.to_numeric(tmp["id"], errors="coerce").astype("Int64")
            tmp = tmp.dropna(subset=["id"]).astype({"id": "int"})
            tmp = tmp.set_index("id")
            out = out.join(tmp, how="left")

    # Sichere Index 1..1000
    out = out.reindex(range(1, 1001))
    return out


def check_credit(df):
    """
    Nimmt den von read_student_surveys erzeugten DataFrame (Index=ID 1..1000).
    Gibt DataFrame mit Spalten:
      - 'name'
      - 'ec' (Extra-Credit-Punkte)
    Regeln:
      - >=50% der Fragen beantwortet -> +5 Punkte (individuell)
      - Mind. 90% der Klasse beantworten eine Frage -> +1 Punkt für alle (max. 2 Fragen zählen)
      - Für Genres-Frage zählt '(no genres listed)' NICHT als valide Antwort.
    """
    out = pd.DataFrame(index=df.index)
    # Name-Spalte übernehmen (falls fehlt, als NA)
    out["name"] = df["name"] if "name" in df.columns else pd.NA

    # Welche Spalten sind Fragen? -> alles außer 'name'
    question_cols = [c for c in df.columns if c != "name"]
    num_questions = len(question_cols)

    # Validitäts-Check pro Frage (nicht-null & nicht nur Leerzeichen)
    valid_matrix = pd.DataFrame(index=df.index)

    for c in question_cols:
        s = df[c]
        # string coercion für Prüfung
        sc = s.astype("string")
        valid = sc.notna() & sc.str.strip().ne("")
        # Spezialfall Genres: Spaltenname enthält 'genre'
        if "genre" in c.lower():
            # '(no genres listed)' zählt nicht
            valid = valid & sc.str.strip().str.lower().ne("(no genres listed)")
        valid_matrix[c] = valid

    # Individuelle 5 Punkte: mind. 50% der Fragen beantwortet
    if num_questions > 0:
        threshold = int(np.ceil(num_questions * 0.5))
        individual5 = (valid_matrix.sum(axis=1) >= threshold).astype(int) * 5
    else:
        individual5 = pd.Series(0, index=df.index)

    # Klassenweit +1 Punkte für jede Frage mit >=90% Antworten (max 2 Fragen)
    # Denominator ist die ganze Klasse (1000 IDs)
    if num_questions > 0:
        answered_counts = valid_matrix.sum(axis=0)  # pro Frage
        classwide_hits = (answered_counts >= 0.9 * 1000).sum()
        classwide_bonus = min(2, int(classwide_hits))
    else:
        classwide_bonus = 0

    out["ec"] = individual5 + classwide_bonus
    return out


In [9]:
# do not edit this cell -- it is needed for the tests
dirname = Path('data') / 'extra-credit-surveys'
q2_out = read_student_surveys(dirname)
check_credit_out = check_credit(q2_out)

In [10]:
grader.check("q2")

q2 results: All test cases passed!

### Question 3 – Paw Patrol 🐾

You are analyzing data from a veterinarian clinic. The datasets contain several types of information from the clinic, including its customers (pet owners), pets, available procedures, and procedure history. The column names are self-explanatory. These DataFrames are provided to you:
-  `owners` stores the customer information, where every `'OwnerID'` is unique (verify this yourself).
-  `pets` stores the pet information. Each pet belongs to a customer in `owners`.
-  `procedure_detail` contains a catalog of procedures that are offered by the clinic.
-  `procedure_history` has procedure records. Most procedures were given to a pet in `pets`.

Complete the implementation of the following three functions, which each ask you to answer a specific question.

#### `most_popular_procedure`

What is the most popular `'ProcedureType'` amongst all pets in the `pets` DataFrame? Complete the implementation of the function `most_popular_procedure`, which takes in two DataFrames, `pets` and `procedure_history`, and returns the name of the most popular `'ProcedureType'` as a string.

Note that some pets are registered but haven't had any procedures performed. Also, some pets that have had procedures done are not registered in `pets`.

<br>

#### `pet_name_by_owner`

What is the name of each customer's pet(s)? Complete the implementation of the function `pet_name_by_owner`, which takes in two DataFrames, `owners` and `pets`, and returns a Series whose index contains owner first names, and whose values are pet names as **strings**. If an owner has multiple pets, the value corresponding to that owner should instead be a **list of pet names as strings**.

Note that owner first names are not necessarily unique, and so the Series you return will not necessarily have a unique index.

<br>

#### `total_cost_per_city`

Note that the `owners` DataFrame has a `'City'` column, describing the city in which each pet owner and their pets live. How much did each city spend in total on procedures? Complete the implementation of the function `total_cost_per_city`, which takes in four DataFrames, `owners`, `pets`, `procedure_history`, and `procedure_detail`, and returns a Series indexed by `'City'` that describes the total amount that each city has spent on pets' procedures.

***Hint***: At some point, you may have to merge on multiple columns.

***Note***: Some owners may have never visited the veterinarian clinic in their city. This means some cities may have zero operational costs.

In [34]:
def most_popular_procedure(pets, procedure_history):
    """
    Gibt den insgesamt am häufigsten vorkommenden ProcedureType als STRING zurück.
    Bei Gleichstand alphabetisch brechen.
    """
    if procedure_history.empty:
        return ""
    counts = (procedure_history.groupby("ProcedureType")
                              .size()
                              .rename("count")
                              .reset_index())
    top = counts.sort_values(["count", "ProcedureType"], ascending=[False, True]).iloc[0]["ProcedureType"]
    return str(top)


def pet_name_by_owner(owners, pets):
    """
    Series: Index = Besitzer-Vorname ('Name' aus owners),
            Wert = alphabetisch, komma-getrennte Liste der Tiernamen.
    Enthält ALLE Besitzer (auch ohne Haustiere -> leerer String).
    Länge == len(owners).
    """
    # Tiernamen je OwnerID sammeln (alphabetisch, komma-getrennt)
    pet_lists = (pets.assign(_pet=pets["Name"].astype(str).str.strip())
                      .dropna(subset=["OwnerID"])
                      .groupby("OwnerID")["_pet"]
                      .apply(lambda s: ", ".join(sorted(s.tolist())))
                      .rename("pet_name"))

    merged = owners.merge(pet_lists, on="OwnerID", how="left")
    merged["pet_name"] = merged["pet_name"].fillna("")

    out = merged.set_index("Name")["pet_name"]
    out.index.name = None
    return out


def total_cost_per_city(owners, pets, procedure_history, procedure_detail):
    """
    Gesamtkosten aller Prozeduren pro Stadt berechnen.
    Rückgabe: Series mit Index = City und Wert = Gesamtpreis (float).
    """
    # Preise an History hängen
    hist = procedure_history.merge(
        procedure_detail[["ProcedureType", "ProcedureSubCode", "Price"]],
        on=["ProcedureType", "ProcedureSubCode"],
        how="left"
    )
    hist["Price"] = pd.to_numeric(hist["Price"], errors="coerce").fillna(0)

    # Über Pet -> Owner -> City auflösen
    hp = hist.merge(pets[["PetID", "OwnerID"]], on="PetID", how="left")
    hpo = hp.merge(owners[["OwnerID", "City"]], on="OwnerID", how="left")

    out = hpo.groupby("City")["Price"].sum()
    out.name = "total_cost"
    return out


In [35]:
# do not edit this cell -- it is needed for the tests
pets_fp = Path('data') / 'pets' / 'Pets.csv'
procedure_history_fp =  Path('data') / 'pets' / 'ProceduresHistory.csv'
owners_fp = Path('data') / 'pets' / 'Owners.csv'
procedure_detail_fp = Path('data') / 'pets' / 'ProceduresDetails.csv'
pets = pd.read_csv(pets_fp)
procedure_history = pd.read_csv(procedure_history_fp)
owners = pd.read_csv(owners_fp)
procedure_detail = pd.read_csv(procedure_detail_fp)

out_01 = most_popular_procedure(pets, procedure_history)
out_02 = pet_name_by_owner(owners, pets)
out_03 = total_cost_per_city(owners, pets, procedure_history, procedure_detail)

In [36]:
grader.check("q3")

q3 results: All test cases passed!

## Part 2: Pivot Tables

### Question 4 – Summarizing Sales 💰

Recall from [Lecture 3](https://dsc80.com/resources/lectures/lec03/lec03-filled.html#Pivot-tables-using-the-pivot_table-method), a pivot table allows you to aggregate the entries in a DataFrame based on two categorical columns. In this question, you are given a simple dataset, `data/sales.csv`, and are asked to solve a few problems using the `pivot_table` method. 

**We have provided outlines for the DataFrames you need to create, but yours may have a different number of rows and columns and different values.**

***Note***: If it helps, [here](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html) is another great resource that provides an overview of `pivot_table` with many examples from the Titanic dataset.


#### `average_seller`

Complete the implementation of the function `average_seller`, which takes in the `sales` DataFrame and returns a DataFrame that contains the average sales for each seller, indexed by `'Name'` and containing the column `'Average Sales'`. There should not be any `NaN`s.

***Note***: You may be able to implement `average_seller` without using `pivot_table`.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Average Sales</th>
    </tr>
    <tr>
      <th>Name</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Jones</th>
      <td>0</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>0</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `product_name`

Complete the implementation of the function `product_name` that takes in the `sales` DataFrame and returns a DataFrame that contains the total sales for each product, indexed by `'Name'`. Do not fill in `NaN`s.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th>Product</th>
      <th>boat</th>
      <th>book</th>
      <th>hotel</th>
    </tr>
    <tr>
      <th>Name</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Jones</th>
      <td>NaN</td>
      <td>0.0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>NaN</td>
      <td>0.0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
  </tbody>
</table>

<br>

#### `count_product`

Complete the implementation of the function `count_product` that takes in the `sales` DataFrame and returns a DataFrame that contains the total number of items sold product-wise and name-wise per date. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th>Date</th>
      <th>01.01.2012</th>
      <th>02.20.2013</th>
      <th>02.25.2015</th>
    </tr>
    <tr>
      <th>Product</th>
      <th>Name</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>boat</th>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">book</th>
      <th>Jones</th>
      <td>0</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>hotel</th>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `total_by_month`

Complete the implementation of the function `total_by_month` that takes in the `sales` DataFrame and returns a pivot table that contains the total sales name-wise, product-wise per month. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th>Month</th>
      <th>February</th>
      <th>January</th>
      <th>July</th>
      <th>March</th>
    </tr>
    <tr>
      <th>Name</th>
      <th>Product</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="3" valign="top">Jones</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">Smith</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

In [22]:
def average_seller(sales):
    """
    DataFrame mit durchschnittlichem Umsatz je Verkäufer.
    Index: 'Name', Spalte: 'Average Sales'. Keine NaNs.
    """
    out = (
        sales.groupby("Name", as_index=True)["Total"]
             .mean()
             .to_frame("Average Sales")
    )
    return out


def product_name(sales):
    """
    Gesamtsumme der Verkäufe je Produkt (Spalten) pro Name (Index).
    NaNs NICHT füllen.
    """
    return sales.pivot_table(
        index="Name",
        columns="Product",
        values="Total",
        aggfunc="sum",
    )


def count_product(sales):
    """
    Anzahl verkaufter Items pro (Product, Name) je Datum.
    NaNs mit 0 ersetzen. Index NICHT zurücksetzen.
    """
    out = sales.pivot_table(
        index=["Product", "Name"],
        columns="Date",
        values="Total",   # beliebige Spalte; wir zählen die Zeilen
        aggfunc="count",
        fill_value=0,
    )
    out.columns.name = "Date"
    return out


def total_by_month(sales):
    """
    Summe der Verkäufe je (Name, Product) pro Monat (Monatsnamen als Spalten).
    NaNs -> 0. Index NICHT zurücksetzen.
    """
    df = sales.copy()
    # Dates sind im Format MM.DD.YYYY (z.B. 02.20.2013)
    df["Month"] = pd.to_datetime(df["Date"], format="%m.%d.%Y").dt.month_name()

    out = df.pivot_table(
        index=["Name", "Product"],
        columns="Month",
        values="Total",
        aggfunc="sum",
        fill_value=0,
    )
    out.columns.name = "Month"
    return out


In [23]:
# don't change this cell -- it is needed for the tests to work
fp = Path('data') / 'sales.csv'
sales = pd.read_csv(fp)
q4_average_seller_out = average_seller(sales)
q4_product_name_out = product_name(sales)
q4_product_count_out = count_product(sales)
q4_total_by_month_out = total_by_month(sales)

In [37]:
grader.check("q4")

q4 results: All test cases passed!

## Congratulations! You're done Lab 3! 🏁

As a reminder, all of the work you want to submit needs to be in `lab.py`.

To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.

Once you've finished the lab, you should open the command line and run, in the directory for this lab:

```
python lab-validation.py
```

**This will run all of the `grader.check` cells that you see in this notebook, but only using the code in `lab.py` – that is, it doesn't look at any of the code in this notebook. If all of your `grader.check` cells pass in this notebook but not all of them pass in your command line with the above command, then you likely have code in your notebook that isn't in your `lab.py`!**

You can also use `lab-validation.py` to test individual questions. For instance,

```
python lab-validation.py q1 q2 q4
```

will run the `grader.check` cells for Questions 1, 2, and 4 – again, only using the code in `lab.py`. [This video](https://www.loom.com/share/0ea254b85b2745e59322b5e5a8692e91?sid=5acc92e6-0dfe-4555-9b6a-8115b6a52f99) how to use the script as well.

Once `python lab-validation.py` shows that you're passing all test cases, you're ready to submit your `lab.py` (and only your `lab.py`) to Gradescope. Once submitting to Gradescope, make sure to stick around until all test cases pass.

There is also a call to `grader.check_all()` below in _this_ notebook, but make sure to also follow the steps above.

<div class="alert alert-block alert-danger" markdown="1">

**There are only two functions in this lab in which you may use a `for`-loop:**
- **`read_linkedin_survey` in Question 1.**
- **`read_student_surveys` in Question 2.**
    
**You may lose points if you use a `for`-loop or `while`-loop in any other question!**

</div>

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()